In [36]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
!pip install spacy
!python -m spacy download zh_core_web_sm
!pip install nltk
!pip install transformers
!pip install sentencepiece
!pip install jieba
!pip nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-05-08 12:51:49.138305: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-08 12:51:50.282710: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Lookin

In [4]:
import json
import spacy
nlp_en = spacy.load("en_core_web_sm")
nlp_zh = spacy.load("zh_core_web_sm")
import sentencepiece
from jieba import posseg
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt", src_lang="zh_CN")


In [39]:
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')
from collections import Counter

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [57]:
class Particularization:

    def __init__(self,path,save_path):
        with open(path) as f:
          text = f.readlines()
        self.en_li = []
        self.zh_li = []
        for i, sent in enumerate(text):
            if i%2 == 0:
                self.en_li.append(sent.strip())
            elif i%2 == 1:
                self.zh_li.append(sent.strip())
        self.save_path = save_path

    def verb(self):
        e = 1
        for en,zh in zip(self.en_li, self.zh_li):
            en_tok_li = [tok_en.text for tok_en in nlp_en(en)]
            if 'put' in en_tok_li and '放' not in zh:
                index = en_tok_li.index('put')+1
                if index < len(en_tok_li) and ''.join([tok.pos_ for tok in nlp_en(en_tok_li[index])]) not in ['ADP','ADV']:

                    print(en)
                    print(zh)
                    print(zh_verb_li)
                    with open(save_path,'a+') as q:
                        q.write(en+'\n'+zh+'\n'+str(zh_verb_li)+'\n'+'put:'+'\n'+'\n')
                        print(e)
                        e += 1


    def ADJ(self):
        r = 1
        for en,zh in zip(self.en_li, self.zh_li):

            en_tok_li = [tok_en.text for tok_en in nlp_en(en)]
            if 'best' in en_tok_li and '好' not in zh and '最' not in zh:

                seg_zh = posseg.cut(zh)

                adj_list = [str(tok).split("/")[0] for tok in seg_zh if str(tok).split("/")[1] == 'a']
                if adj_list != []:
                    
                    
                    print(en)
                    print(zh)
                    print(adj_list)
                    print()
                    with open(save_path,'a+') as q:
                        q.write(en+'\n'+zh+'\n'+str(adj_list)+'\n'+'large:'+'\n')
                        print(r)
                        r += 1

    def NOUN(self):
        r = 1
        for en,zh in zip(self.en_li, self.zh_li):

            en_tok_li = [tok_en.text for tok_en in nlp_en(en)]
            if 'group' in en_tok_li and '团' not in zh and '体' not in zh and '组' not in zh and 'group of' not in en and 'groups of' not in en :

                seg_zh = posseg.cut(zh)

                noun_list = [str(tok).split("/")[0] for tok in seg_zh if str(tok).split("/")[1] == 'n']
                if noun_list != []:
                    
                    print(en)
                    print(zh)
                    print(noun_list)
                    print()
                    with open(save_path,'a+') as q:
                        q.write(en+'\n'+zh+'\n'+str(noun_list)+'\n'+'group:'+'\n')
                        print(r)
                        r += 1


    def PRONOUN(self):
        r = 1
        for en,zh in zip(self.en_li, self.zh_li):

            en_tok_li = [tok_en.text for tok_en in nlp_en(en)]
            if 'all' in en_tok_li and 'at all' not in en and 'after all' not in en: #and '所有' not in zh and '全部' not in zh and '全' not in zh and '都' not in zh and '一' not in zh and 'at all' not in en: #and "it's" not in en and "it comes" not in en and "that" not in en:
                # if en_tok_li.index('it') != 0 and "".join(tok.pos_ for tok in nlp_en(en_tok_li[en_tok_li.index('it')-1])) in ['VERB','ADV']:
                    
                seg_zh = posseg.cut(zh)
                noun_list = [str(tok).split("/")[0] for tok in seg_zh if tok.flag == 'nr']
                if noun_list != []:
                    print(en)
                    print(zh)
                    print(noun_list)
                    print()
                    # with open(save_path,'a+') as q:
                    #     q.write(en+'\n'+zh+'\n'+str(noun_list)+'\n'+'all:'+'\n')   
                    #     print(r)
                    #     r += 1



    def pronoun_particular(self,pron = ' it ',exception_zh = [],exception_en = []):
        for en,zh in zip(self.en_li, self.zh_li):
            if pron in en:
                if len([en_tok.text for en_tok in nlp_en(en).ents if en_tok.label_=='PERSON']) == 0 and len([en_tok.text for en_tok in nlp_en(en) if en_tok.pos_=='n']) == 0:
                    i = 0 
                    j = 0
                    for excep in exception_zh:
                        if excep not in zh:
                            i += 1
                    for excep_en in exception_en:
                        if excep_en not in en:
                            j += 1
                    if i == len(exception_zh) and j == len(exception_en):
                        if len([str(zh_tok).split('/')[0] for zh_tok in posseg.cut(zh) if str(zh_tok).split('/')[-1] =='nr']) != 0:
                            print(en)
                            print(zh)
                            
                            print([str(zh_tok).split('/')[0] for zh_tok in posseg.cut(zh) if str(zh_tok).split('/')[-1] =='nr'])
                            print('--------')
                            with open(self.save_path,'a+') as y:
                                y.write(en+'\n'+zh+'\n'+pron.strip()+':'+'\n'+'\n')


    def pronoun_particular_which(self,pron = ' it ',exception_zh = [],exception_en = []):
        for en,zh in zip(self.en_li, self.zh_li):
            if pron in en:
                if len([en_tok.text for en_tok in nlp_en(en).ents if en_tok.label_=='PERSON']) == 0 and len([en_tok.text for en_tok in nlp_en(en) if en_tok.pos_=='n']) == 0:
                    i = 0 
                    j = 0
                    for excep in exception_zh:
                        if excep not in zh:
                            i += 1
                    for excep_en in exception_en:
                        if excep_en not in en:
                            j += 1
                    if i == len(exception_zh) and j == len(exception_en):
                        if len([str(zh_tok).split('/')[0] for zh_tok in posseg.cut(zh) if str(zh_tok).split('/')[-1] =='nr']) != 0:
                            zh_ele_li = [str(zh_tok).split('/')[0] for zh_tok in posseg.cut(zh) if str(zh_tok).split('/')[-1] =='nr']
                            my_counter = dict(Counter(zh_ele_li))

                            list_word_right = []
                            for k, v in my_counter.items():
                                if v==2:
                                    list_word_right.append(k)
                                    print(en)
                                    print(zh)
                                    print(k)
                
                                    print('--------')
                            for ele in list(set(list_word_right)):
                                with open(self.save_path,'a+') as y:
                                    y.write(en+'\n'+zh+'\n'+pron.strip()+':'+ele+'\n'+'\n')
            


      
spath = '/content/gdrive/MyDrive/Colab Notebooks/literal translationese/dataset/Bilingual/Microblog/Bi-Microblog.txt'
save_path = '/content/gdrive/MyDrive/Colab Notebooks/literal translationese/dataset/feature45_particular_generalization/pronoun_new_particular_which_new.txt'             
        
ptcl = Particularization(spath, save_path)

ptcl.pronoun_particular_which(pron=' which ',exception_zh = ['是','她','们','这','他','它','这','人','谁'],exception_en = [])

                 